This tutorial describes how to evaluate new rules applicable to the derivatives data.

In [ ]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator 

In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile
import re
from src import Evaluator
import logging
import data_patterns

In [ ]:
DECIMALS = 0
RULES_PATH = join('..', 'solvency2-rules')
INSTANCES_DATA_PATH = join('..', 'tests', 'data')
FILENAME_DATAPOINTS = 'QRS.csv'
RESULTS_PATH = join('..', 'results')
DATA_PATH = join('..', 'data')
logging.basicConfig(filename = join(RESULTS_PATH, 'rules.log'),level = logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Import rules

We start with importing the new rules applicable to the derivatives data. One set of rules is applicable to template S2.08.01.01 (Open derivatives). This template consists of both S2.08.01.01.01 (Information on positions held) and S2.08.01.01.02 (Information on derivatives). The other set of rules is only applicable to S2.08.01.01.02 (Information on derivatives).

In [ ]:
dfr = pd.read_excel(join(RULES_PATH,'S2_08_01_01.xlsx'), index_col='index')
dfr2 = pd.read_excel(join(RULES_PATH,'S2_08_01_01_02.xlsx'), index_col='index')

#Capitalize row-column references:
column_replace = set([column for sublist in [row for row in dfr['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr2['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr2.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)   

### Import templates

Next we import the reporting data. In the tutorial 'Convert Solvency 2 XBRL-instances to CSV, HTML and pickles' the XBRL-instances are converted to pickle files per template. These pickle files are written to the data/instances folder.  We are going to import these pickle files. In this tutorial we use dummy pickle files that are in the tests/data folder of this repository. 

We create two dataframes with reporting data. For the set of rules that is applicable to template S2.08.01.01 (Open derivatives) we merge S2.08.01.01.01 (Information on positions held) and S2.08.01.01.02 (Information on derivatives). The other set of rules is only applicable to S2.08.01.01.02 (Information on derivatives).

For the sake of simplicity we only import the Quarterly Solvency II reporting Solo (QRS) templates.

In [ ]:
df1 = pd.read_pickle(join(INSTANCES_DATA_PATH,'S.08.01.01.01.pickle')).reset_index()
df2 = pd.read_pickle(join(INSTANCES_DATA_PATH, 'S.08.01.01.02.pickle')).reset_index()
df = pd.merge(df1,df2,how='inner', left_on=['entity','period','S.08.01.01.01,C0040'], right_on=['entity','period','S.08.01.01.02,C0040']).set_index(['entity', 'period', 'S.08.01.01.01,C0040'])
df2 = df2.set_index(['entity', 'period', 'S.08.01.01.02,C0040'])

In [ ]:
df

### Evaluate rules

Now we are ready to evaluate the rules. First, we construct a PatternMiner-object with the data-patterns package using the rules dataframe. Second, we use the analyze-function to get the results of the rules. We do this for both set of rules separately. 

In [ ]:
miner = data_patterns.PatternMiner(df_patterns=dfr)
results = miner.analyze(df)
results

In [ ]:
miner = data_patterns.PatternMiner(df_patterns=dfr2)
results2 = miner.analyze(df2)
results2